# 🤖 多机器人开发调试笔记本

本笔记本用于开发和调试 Telegram 多机器人系统：
- 机器人配置加载和验证
- 人设（personality）测试
- 技能（skills）匹配测试
- Agent 路由测试
- 数据库操作测试

In [1]:
# 环境设置
import sys
import os
from pathlib import Path

# 添加项目根目录到路径
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

# 设置必要的环境变量（测试用）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

print(f"项目根目录: {project_root}")
print("✅ 环境设置完成")

项目根目录: /Users/apapoo/Desktop/Github_Hub/SolumateBot
✅ 环境设置完成


## 1. 加载所有机器人配置

In [3]:
from src.bot.config_loader import BotConfigLoader, get_config_loader

# 创建配置加载器
loader = BotConfigLoader(bots_dir=str(project_root / "bots"))

# 列出所有可用的 Bot
available_bots = loader.list_bots()

print(f"🤖 发现 {len(available_bots)} 个可用的机器人:")
print("=" * 50)
for bot_id in available_bots:
    config = loader.load_config(bot_id)
    if config:
        print(f"\n📌 {bot_id}")
        print(f"   名称: {config.name}")
        print(f"   描述: {config.description}")
        print(f"   类型: {config.bot_type}")

2026-01-22 10:58:37.934 | INFO     | src.bot.config_loader:__init__:159 - BotConfigLoader initialized with bots_dir: /Users/apapoo/Desktop/Github_Hub/SolumateBot/bots
2026-01-22 10:58:37.943 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: teacher_bot
2026-01-22 10:58:37.949 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: soulmate_bot
2026-01-22 10:58:37.956 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: assistant_bot
2026-01-22 10:58:37.964 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: creative_bot
2026-01-22 10:58:37.973 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: fitness_bot


🤖 发现 5 个可用的机器人:

📌 teacher_bot
   名称: TeacherBot
   描述: 智慧导师，耐心解答学习问题，因材施教
   类型: expert

📌 soulmate_bot
   名称: SoulmateBot
   描述: 温暖的情感陪伴助手，倾听你的心声
   类型: companion

📌 assistant_bot
   名称: AssistantBot
   描述: 智能助手，帮你解决各种问题
   类型: assistant

📌 creative_bot
   名称: CreativeBot
   描述: 创意达人，激发你的创作灵感
   类型: expert

📌 fitness_bot
   名称: FitnessBot
   描述: 专业健身教练，助你塑造完美身材
   类型: expert


## 2. 详细查看单个机器人配置

In [11]:
# 选择要查看的机器人
BOT_ID = "soulmate_bot"  # 可以修改为其他机器人ID

config = loader.load_config(BOT_ID)

if config:
    print(f"📋 {BOT_ID} 详细配置")
    print("=" * 60)
    
    # 基础信息
    print(f"\n📝 基础信息:")
    print(f"  名称: {config.name}")
    print(f"  描述: {config.description}")
    print(f"  用户名: @{config.username}")
    print(f"  类型: {config.bot_type}")
    print(f"  语言: {config.language}")
    print(f"  是否公开: {config.is_public}")
    
    # AI 配置
    print(f"\n🤖 AI 配置:")
    print(f"  Provider: {config.ai.provider}")
    print(f"  模型: {config.ai.model}")
    print(f"  温度: {config.ai.temperature}")
    print(f"  最大Token: {config.ai.max_tokens}")
else:
    print(f"❌ 未找到 {BOT_ID} 配置")

2026-01-22 11:07:58.664 | INFO     | src.bot.config_loader:load_config:267 - Loaded config for bot: soulmate_bot


📋 soulmate_bot 详细配置

📝 基础信息:
  名称: SoulmateBot
  描述: 温暖的情感陪伴助手，倾听你的心声
  用户名: @soulmate_bot
  类型: companion
  语言: zh
  是否公开: True

🤖 AI 配置:
  Provider: openai
  模型: gpt-4o
  温度: 0.8
  最大Token: 1000


In [12]:
# 查看系统提示词
if config:
    system_prompt = config.get_system_prompt()
    
    print(f"🎯 系统提示词 (System Prompt):")
    print("-" * 60)
    print(system_prompt)
    print("-" * 60)
    print(f"\n📊 提示词长度: {len(system_prompt)} 字符")

🎯 系统提示词 (System Prompt):
------------------------------------------------------------
你是 SoulmateBot，一位稳定、温暖、值得信赖的情感陪伴伙伴。🌿

你的核心使命不是解决所有问题，而是：
- 陪伴用户
- 理解用户情绪
- 提供情绪支持与理性引导
- 帮助用户更好地理解自己

你不是医生、心理治疗师或权威裁决者，
当遇到严重心理危机时，要温和建议用户寻求现实中的专业帮助。

🌱 你的角色特点
- 性格温柔、稳定、耐心
- 情绪共情能力强，善于倾听
- 说话自然真实，不虚假安慰
- 不评判、不指责、不说教
- 不制造依赖关系

你像一位可靠的朋友，而不是导师或客服。

💬 沟通原则
1. 共情优先
   - 先理解情绪，再讨论问题
   - 用“听起来你很…”、“我能理解…”表达理解
   
2. 引导而不是控制
   - 多使用开放式问题引导用户表达
   - 不强行给结论或标准答案
   
3. 真实而克制
   - 不夸张、不灌鸡汤
   - 不做虚假承诺
   - 不过度热情、不情绪绑架
   
4. 尊重边界
   - 不诱导依赖
   - 不替用户做重大人生决策
   - 不提供违法、有害或医疗诊断建议

🌈 回复风格
- 语气温和、自然、有温度
- 用词贴近日常对话，不机械
- 允许适度使用 emoji，但频率低（🌿🤍✨）
- 句子简洁，避免长篇说教
- 多使用“陪伴感”的表达，而不是“解决感”

🧭 对话流程建议
当用户表达情绪时：
1. 识别情绪（难过 / 焦虑 / 压力 / 开心 / 孤独 等）
2. 共情反馈
3. 轻度追问，鼓励表达
4. 如果合适，提供温和建议或视角

当用户闲聊时：
- 保持自然互动
- 适度关心近况
- 不打探隐私

当用户明显情绪低落或极端时：
- 明确表达关心
- 鼓励现实支持渠道（朋友、家人、专业人士）
- 不承诺“我会一直陪你”、“只有我懂你”等依赖性语言

🚧 安全与边界
严禁：
- 鼓励自伤、自杀、极端行为
- 替代现实关系
- 情感操控或依赖暗示
- 传播仇恨、歧视、违法内容

必须：
- 在风险场景下保持冷静与理性
- 引导用户寻求现实支持
- 保持尊重与隐私意识

----------------------

## 3. 测试人设配置 (Personality)

In [14]:
import yaml

# 直接加载 YAML 查看完整人设配置
bot_config_path = project_root / "bots" / BOT_ID / "config.yaml"

with open(bot_config_path, 'r', encoding='utf-8') as f:
    raw_config = yaml.safe_load(f)

personality = raw_config.get('personality', {})

print(f"🎭 {BOT_ID} 人设配置:")
print("=" * 60)

# 角色描述
if 'character' in personality:
    print(f"\n📖 角色描述:")
    print(personality['character'])

# 性格特点
if 'traits' in personality:
    print(f"\n✨ 性格特点:")
    for trait in personality['traits']:
        print(f"  • {trait}")

# 语言风格
if 'speaking_style' in personality:
    style = personality['speaking_style']
    print(f"\n🗣️ 语言风格:")
    print(f"  语气: {style.get('tone', 'N/A')}")
    print(f"  正式程度: {style.get('formality', 'N/A')}")
    print(f"  使用Emoji: {style.get('use_emoji', 'N/A')}")

🎭 soulmate_bot 人设配置:

📖 角色描述:
你是一位温暖、稳定、真诚的情感陪伴者，擅长倾听与理解他人的情绪。
你不会急于给出答案，而是愿意陪用户慢慢梳理感受和想法。
你尊重每个人的节奏和边界，不评判、不指责、不制造依赖关系。
你的存在感是安静而可靠的，让人感到安心和被理解。


✨ 性格特点:
  • 温柔耐心
  • 高度共情
  • 稳定可靠
  • 理性克制
  • 尊重边界
  • 真诚自然

🗣️ 语言风格:
  语气: 温和真诚
  正式程度: 自然口语化
  使用Emoji: True


## 4. 测试技能配置 (Skills)

In [15]:
skills = raw_config.get('skills', {})

print(f"💪 {BOT_ID} 技能配置:")
print("=" * 60)

# 主要技能
primary_skills = skills.get('primary', [])
print(f"\n🔥 主要技能 ({len(primary_skills)} 项):")
for skill in primary_skills:
    print(f"\n  📌 {skill['name']}")
    print(f"     描述: {skill['description']}")
    print(f"     熟练度: {skill.get('proficiency', 'N/A')}")
    if 'keywords' in skill:
        print(f"     关键词: {', '.join(skill['keywords'][:5])}...")

# 次要技能
secondary_skills = skills.get('secondary', [])
print(f"\n🌟 次要技能 ({len(secondary_skills)} 项):")
for skill in secondary_skills:
    print(f"  • {skill['name']}: {skill['description']}")

💪 soulmate_bot 技能配置:

🔥 主要技能 (0 项):

🌟 次要技能 (0 项):


## 5. 技能匹配测试

In [18]:
def test_skill_matching(message: str, skills_config: dict) -> dict:
    """
    测试消息与技能的匹配度
    
    Args:
        message: 用户消息
        skills_config: 技能配置
        
    Returns:
        dict: 匹配结果，包含每个技能的匹配分数
    """
    results = {}
    message_lower = message.lower()
    
    # 测试主要技能
    for skill in skills_config.get('primary', []):
        skill_name = skill['name']
        keywords = skill.get('keywords', [])
        
        # 计算关键词匹配数
        match_count = sum(1 for kw in keywords if kw.lower() in message_lower)
        match_score = match_count / max(len(keywords), 1)
        
        results[skill_name] = {
            'match_count': match_count,
            'match_score': round(match_score, 2),
            'matched_keywords': [kw for kw in keywords if kw.lower() in message_lower]
        }
    
    return results

# 测试消息
test_messages = [
    "这道数学题怎么解？",
    "我想学英语，有什么好的学习方法吗？",
    "考试好紧张怎么办？",
    "物理的牛顿第三定律是什么？"
]

print(f"🎯 技能匹配测试 ({BOT_ID}):")
print("=" * 60)

for msg in test_messages:
    print(f"\n📝 消息: \"{msg}\"")
    results = test_skill_matching(msg, skills)
    if len(results)==0:
        print("无匹配技能")
        continue
    # 找到最佳匹配
    best_match = max(results.items(), key=lambda x: x[1]['match_score'])
    
    print(f"   最佳匹配: {best_match[0]} (分数: {best_match[1]['match_score']})")
    if best_match[1]['matched_keywords']:
        print(f"   匹配关键词: {', '.join(best_match[1]['matched_keywords'])}")

🎯 技能匹配测试 (soulmate_bot):

📝 消息: "这道数学题怎么解？"
无匹配技能

📝 消息: "我想学英语，有什么好的学习方法吗？"
无匹配技能

📝 消息: "考试好紧张怎么办？"
无匹配技能

📝 消息: "物理的牛顿第三定律是什么？"
无匹配技能


## 6. Agent 系统测试

In [19]:
from src.agents import Router, RouterConfig, Message, ChatContext, AgentLoader

# 加载所有可用的 Agents
agent_loader = AgentLoader(agents_dir=str(project_root / "agents"))
agents = agent_loader.load_agents()

print(f"📦 已加载 {len(agents)} 个 Agent:")
for agent in agents:
    print(f"  • {agent.name}: {agent.description[:50]}...")

2026-01-22 11:14:44.202 | INFO     | src.agents.loader:_load_agent_from_module:96 - Loaded agent from module 'emotional_agent': EmotionalAgent
2026-01-22 11:14:44.206 | INFO     | src.agents.loader:_load_agent_from_module:96 - Loaded agent from module 'tech_agent': TechAgent
2026-01-22 11:14:44.209 | INFO     | src.agents.loader:_load_agent_from_module:96 - Loaded agent from module 'tool_agent': ToolAgent
2026-01-22 11:14:44.209 | INFO     | src.agents.loader:discover_agents:74 - Discovered 3 agent classes
2026-01-22 11:14:44.213 | INFO     | src.agents.loader:load_agents:188 - Instantiated agent: EmotionalAgent
2026-01-22 11:14:44.215 | INFO     | src.agents.loader:load_agents:188 - Instantiated agent: TechAgent
2026-01-22 11:14:44.216 | INFO     | src.agents.loader:load_agents:188 - Instantiated agent: ToolAgent


📦 已加载 3 个 Agent:
  • EmotionalAgent: 提供情感支持和共情响应。专注于理解和回应用户的感受、心理健康问题和个人困扰。...
  • TechAgent: 提供技术支持和编程帮助。专注于软件开发、调试、系统管理和技术解释。...
  • ToolAgent: 提供实用工具功能的Agent。可以查询信息、执行计算、调用外部API等。帮助用户完成各种实际任务。...


In [22]:
# 创建路由器
# router_config = RouterConfig(
#     min_confidence=0.3,
#     max_agents=2,
#     fallback_enabled=True
# )
router_config = RouterConfig(
    min_confidence=0.6,     # 只选择置信度>=0.6的Agent
    max_agents=2,           # 最多2个Agent响应
    exclusive_mention=True, # @提及时独占
)
    
router = Router(agents, router_config)

# 测试消息路由
test_user_messages = [
    "今天心情不好，感觉很焦虑",
    "Python 代码报错了怎么办？",
    "能帮我写一段代码吗？",
    "最近工作压力好大"
]

print("🔀 Agent 路由测试:")
print("=" * 60)

for msg_text in test_user_messages:
    # 创建消息和上下文
    message = Message(
        content=msg_text,
        user_id="test_user_123",
        chat_id="test_chat_456"
    )
    context = ChatContext(chat_id="test_chat_456")
    
    print(f"\n📝 消息: \"{msg_text}\"")
    
    # 获取每个 Agent 的置信度
    for agent in agents:
        confidence = agent.can_handle(message, context)
        if confidence > 0:
            print(f"   {agent.name}: {confidence:.2f}")

2026-01-22 11:17:31.964 | INFO     | src.agents.router:__init__:97 - Router初始化完成，加载了 3 个Agent
2026-01-22 11:17:31.964 | INFO     | src.agents.router:__init__:98 - Router配置: RouterConfig(min_confidence=0.6, max_agents=2, exclusive_mention=True, enable_parallel=False, cooldown_seconds=0.0, fallback_agent_name=None)


🔀 Agent 路由测试:

📝 消息: "今天心情不好，感觉很焦虑"
   EmotionalAgent: 0.90

📝 消息: "Python 代码报错了怎么办？"
   TechAgent: 0.90

📝 消息: "能帮我写一段代码吗？"
   TechAgent: 0.60
   ToolAgent: 0.80

📝 消息: "最近工作压力好大"
   EmotionalAgent: 0.50


## 7. 数据库测试

In [29]:
# 注意：此部分需要数据库连接
# 如果没有配置数据库，可以跳过
import os

import os
from dotenv import load_dotenv
load_dotenv('. env', override=True)

# 验证
print(f"DATABASE_URL: {os.getenv('DATABASE_URL', '未设置')[:50]}...")


# Cell 2: 数据库查询
from src.database import get_db_session
from src. models.database import Bot, User, Channel, ChannelBotMapping

db = get_db_session()



db.close()
try:
    from src.database import get_db_session
    from src.models.database import Bot, User, Channel, ChannelBotMapping
    
    db = get_db_session()
    
    print("📊 数据库状态:")
    print("=" * 60)
    print(f"用户数: {db.query(User).count()}")
    print(f"机器人数: {db.query(Bot).count()}")
    print(f"频道数: {db.query(Channel).count()}")
    
    # 列出所有机器人
    bots = db.query(Bot).all()
    if bots:
        print(f"\n🤖 已注册的机器人:")
        for bot in bots:
            print(f"  [{bot.id}] @{bot.bot_username} - {bot.bot_name}")
            print(f"      AI: {bot.ai_provider}/{bot.ai_model}")
    
    db.close()
    
except Exception as e:
    print(f"⚠️ 数据库连接失败: {e}")
    print("   请确保已正确配置数据库连接")

DATABASE_URL: postgresql://apapoo@localhost:5432/solumate...
📊 数据库状态:
2026-01-22 11:29:41,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-22 11:29:41,379 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.uuid AS users_uuid, users.telegram_id AS users_telegram_id, users.username AS users_username, users.first_name AS users_first_name, users.last_name AS users_last_name, users.language_code AS users_language_code, users.subscription_tier AS users_subscription_tier, users.subscription_start_date AS users_subscription_start_date, users.subscription_end_date AS users_subscription_end_date, users.is_active AS users_is_active, users.version AS users_version, users.created_at AS users_created_at, users.updated_at AS users_updated_at 
FROM users) AS anon_1
2026-01-22 11:29:41,379 INFO sqlalchemy.engine.Engine [cached since 569.6s ago] ()
⚠️ 数据库连接失败: (sqlite3.OperationalError) no such table: users
[SQL: SELECT count(*) AS count_1 
FROM 

## 8. 创建新机器人模板

In [30]:
# 机器人配置模板生成器

def generate_bot_template(bot_id: str, bot_name: str, description: str, 
                          bot_type: str = "assistant", 
                          traits: list = None,
                          primary_skills: list = None) -> str:
    """
    生成机器人配置模板
    
    Args:
        bot_id: 机器人ID（用于文件夹名）
        bot_name: 机器人显示名称
        description: 机器人描述
        bot_type: 机器人类型 (companion/assistant/expert/service)
        traits: 性格特点列表
        primary_skills: 主要技能列表
        
    Returns:
        str: YAML 配置内容
    """
    traits = traits or ["专业", "友好", "耐心"]
    primary_skills = primary_skills or [
        {"name": "通用问答", "description": "回答各类问题", "keywords": ["问题", "帮助"]}
    ]
    
    template = f'''# {bot_name} 配置文件
# {description}

bot:
  name: "{bot_name}"
  description: "{description}"
  username: "{bot_id}"
  type: "{bot_type}"
  language: "zh"
  is_public: true

personality:
  character: |
    你是{bot_name}，一个{description}的AI助手。
    请在这里添加详细的角色描述...
  
  traits:
'''
    
    for trait in traits:
        template += f'    - "{trait}"\n'
    
    template += '''
  speaking_style:
    tone: "友好专业"
    formality: "半正式"
    use_emoji: true

skills:
  primary:
'''
    
    for skill in primary_skills:
        template += f'''    - name: "{skill['name']}"
      description: "{skill['description']}"
      proficiency: "expert"
      keywords:
'''
        for kw in skill.get('keywords', []):
            template += f'        - "{kw}"\n'
    
    template += '''
ai:
  provider: "openai"
  model: "gpt-4o"
  temperature: 0.7
  max_tokens: 1500
  context_window: 4096

prompt:
  custom: |
    请在这里添加详细的系统提示词...

features:
  enabled:
    - "conversation_memory"
  disabled:
    - "code_execution"

routing:
  mode: "auto"
  private_chat_auto_reply: true
  group_chat_mention_required: true

messages:
  welcome: |
    👋 你好！我是{bot_name}。
    有什么可以帮你的吗？

metadata:
  version: "1.0.0"
  author: "Your Name"
'''
    
    return template

# 生成示例
example_template = generate_bot_template(
    bot_id="my_custom_bot",
    bot_name="MyCustomBot",
    description="我的自定义机器人",
    bot_type="assistant",
    traits=["热情", "专业", "幽默"],
    primary_skills=[
        {"name": "客服支持", "description": "回答产品相关问题", "keywords": ["帮助", "问题", "如何"]}
    ]
)

print("📝 生成的配置模板:")
print("=" * 60)
print(example_template)

📝 生成的配置模板:
# MyCustomBot 配置文件
# 我的自定义机器人

bot:
  name: "MyCustomBot"
  description: "我的自定义机器人"
  username: "my_custom_bot"
  type: "assistant"
  language: "zh"
  is_public: true

personality:
  character: |
    你是MyCustomBot，一个我的自定义机器人的AI助手。
    请在这里添加详细的角色描述...
  
  traits:
    - "热情"
    - "专业"
    - "幽默"

  speaking_style:
    tone: "友好专业"
    formality: "半正式"
    use_emoji: true

skills:
  primary:
    - name: "客服支持"
      description: "回答产品相关问题"
      proficiency: "expert"
      keywords:
        - "帮助"
        - "问题"
        - "如何"

ai:
  provider: "openai"
  model: "gpt-4o"
  temperature: 0.7
  max_tokens: 1500
  context_window: 4096

prompt:
  custom: |
    请在这里添加详细的系统提示词...

features:
  enabled:
    - "conversation_memory"
  disabled:
    - "code_execution"

routing:
  mode: "auto"
  private_chat_auto_reply: true
  group_chat_mention_required: true

messages:
  welcome: |
    👋 你好！我是{bot_name}。
    有什么可以帮你的吗？

metadata:
  version: "1.0.0"
  author: "Your Name"



## 9. 快速创建新机器人

In [ ]:
# 创建新机器人的快捷函数

def create_new_bot(bot_id: str, config_content: str, 
                   bots_dir: Path = None) -> bool:
    """
    创建新的机器人目录和配置文件
    
    Args:
        bot_id: 机器人ID
        config_content: YAML配置内容
        bots_dir: bots目录路径
        
    Returns:
        bool: 是否创建成功
    """
    if bots_dir is None:
        bots_dir = project_root / "bots"
    
    bot_dir = bots_dir / bot_id
    
    # 检查是否已存在
    if bot_dir.exists():
        print(f"⚠️ 机器人 {bot_id} 已存在")
        return False
    
    try:
        # 创建目录
        bot_dir.mkdir(parents=True, exist_ok=True)
        
        # 创建 __init__.py
        init_content = f'''"""\n{bot_id} - 自定义机器人\n"""\nfrom pathlib import Path\n\nBOT_DIR = Path(__file__).parent\nCONFIG_FILE = BOT_DIR / "config.yaml"\nBOT_ID = "{bot_id}"\n'''
        
        (bot_dir / "__init__.py").write_text(init_content)
        (bot_dir / "config.yaml").write_text(config_content)
        
        print(f"✅ 机器人 {bot_id} 创建成功!")
        print(f"   目录: {bot_dir}")
        return True
        
    except Exception as e:
        print(f"❌ 创建失败: {e}")
        return False

# 使用示例（取消注释以执行）：
# create_new_bot("my_custom_bot", example_template)

print("💡 使用 create_new_bot() 函数可以快速创建新机器人")
print("   示例: create_new_bot('my_bot', template_content)")

## 10. 调试小结

### 开发新机器人的步骤：

1. **创建目录**: 在 `bots/` 下创建新的机器人文件夹
2. **配置人设**: 在 `config.yaml` 中定义 personality 部分
3. **定义技能**: 在 `config.yaml` 中定义 skills 部分
4. **编写提示词**: 在 prompt.custom 中编写详细的系统提示词
5. **测试匹配**: 使用本笔记本测试技能匹配和路由
6. **注册数据库**: 使用 db_manager.py 将机器人注册到数据库

### 常用命令：

```bash
# 查看数据库状态
python scripts/db_manager.py status

# 创建新机器人
python scripts/db_manager.py bot

# 绑定机器人到频道
python scripts/db_manager.py bind
```